**Classifier**

Classification involves predicting what class something belongs to. Classifiers can be binary or multi-class, meaning that they either classify something into a binary decision ( yes/no, spam/not spam, hot/cold ) or into several different categories ( blue, yellow, red or green? ). Classification models are a very common use case in deep learning, and they can be used to solve a lot of different problems. 

Now, we are going to build a classification model that can help us to classify some flower photographs into three different species.

In [2]:
import numpy as np

import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report

from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import np_utils

Using TensorFlow backend.


**The Data**

The Iris flower dataset or Fisher's Iris dataset is a multivariate dataset that quantifies the morphologic variation of Iris flowers of three related species.

1. Sepal length in cm
2. Sepal width in cm
3. Petal length in cm
4. Petal width in cm
5. Classes:
  - Iris Setosa
  - Iris Versicolour
  - Iris Virginica

  ![Iris](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Machine+Learning+R/iris-machinelearning.png)


In [11]:
# Download the data
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data -O ../Datasets/iris-data.csv

--2020-02-19 08:23:25--  http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551 (4.4K) [application/x-httpd-php]
Saving to: ‘../Datasets/iris-data.csv’

../Datasets/iris-da 100%[===================>]   4.44K  --.-KB/s    in 0s      

2020-02-19 08:23:26 (459 MB/s) - ‘../Datasets/iris-data.csv’ saved [4551/4551]



**Loading a csv file**

This data is organized inside a csv file. We are going to read the values from this file and organize them into arrays.

In [13]:
import os.path
path = "Datasets/iris-data.csv"
if os.path.isfile("../" + path) :
    dataset = pd.read_csv("../" + path,header=None).values
else:
    dataset = pd.read_csv("2-Basic-neural-networks/" + path , header=None).values

print("The dataset examples:")
print(dataset[0])
print(dataset[50])
print(dataset[100])

The dataset examples:
[5.1 3.5 1.4 0.2 'Iris-setosa']
[7.0 3.2 4.7 1.4 'Iris-versicolor']
[6.3 3.3 6.0 2.5 'Iris-virginica']


**Separating the inputs and outputs**

This dataset need to be organized and preprocessed before it can be used inside the neural network. Let's start by separating the input values to an array (X) and the output values to another array (Y).

In [8]:
# Getting the input data
X = dataset[:,0:4].astype(float)
# Getting the output
Y = dataset[:,4]

print("The dataset examples:")
print("Input:",X[0],"Output:",Y[0])
print("Input:",X[50],"Output:",Y[50])
print("Input:",X[100],"Output:",Y[100])

The dataset examples:
Input: [5.1 3.5 1.4 0.2] Output: Iris-setosa
Input: [7.  3.2 4.7 1.4] Output: Iris-versicolor
Input: [6.3 3.3 6.  2.5] Output: Iris-virginica


**Encoding the labels**

Remember that neural networks are mathematical functions but our labels are names. We also need to convert the way we represent those names to a numerical form.

Since we have three species of flowers, so each one of them will be represented by three numbers. Each one of these numbers will also represent an output of the neural network, so we are going to have three outputs. This means that we want only one of the outputs neurons to activate for each class we have.

In [9]:
# Change the classes to numerical values
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
Y = np_utils.to_categorical(encoded_Y)

print("The dataset examples:")
print("Iris-setosa")
print("Input:",X[0],"Output:",Y[0])
print("Iris-versicolor")
print("Input:",X[50],"Output:",Y[50])
print("Iris-virginica")
print("Input:",X[100],"Output:",Y[100])

The dataset examples:
Iris-setosa
Input: [5.1 3.5 1.4 0.2] Output: [1. 0. 0.]
Iris-versicolor
Input: [7.  3.2 4.7 1.4] Output: [0. 1. 0.]
Iris-virginica
Input: [6.3 3.3 6.  2.5] Output: [0. 0. 1.]


**Separating the train and test data**

The network need to be able to perform with data that it hasn't seen during the training process. So we separate a small portion of the dataset out of the training process to better evaluate its performance later.

In [10]:
# Shuffle the dataset
X, Y = shuffle(X, Y, random_state=0)

# 80% for training and 20% for test
p = int(len(dataset)* 0.8 )

x_train = X[:p]
x_test = X[p:]

y_train = Y[:p]
y_test = Y[p:]

**Building a regression model**

* Input layer: 4 neurons (we have 4 inputs)
* Hidden 2 hidden layers of 6 neurons each.
* Output layer: 3 neuron (we have 3 classes we are trying to classify).
* Activation functions: Here we are going to introduce the softmax activation function.

In [11]:
def build_model():
  # Input layer
  inputs = Input([4, ], name='data')
  # Hidden layers
  model = Dense(6, activation='relu', name='fc1')(inputs)
  model = Dense(6, activation='relu', name='fc2')(model)
  # Output layer
  outputs = Dense(3, activation='softmax', name='fc3')(model)

  # Define the model
  model = Model(inputs=[inputs], outputs=outputs)

  model.compile(optimizer='adam',#Adam(lr=1e-5),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model

net = build_model()
print(net.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
data (InputLayer)            (None, 4)                 0         
_________________________________________________________________
fc1 (Dense)                  (None, 6)                 30        
_________________________________________________________________
fc2 (Dense)                  (None, 6)                 42        
_________________________________________________________________
fc3 (Dense)                  (None, 3)                 21        
Total params: 93
Trainable params: 93
Non-trainable params: 0
_________________________________________________________________
None


**Training the model**

Now we have our data separated into inputs and outputs, and training data and testing data. We also have defined our architecture. All that is left to do is fit the data inside our model (train).

In [15]:
net.fit(x_train,
        y_train,
        validation_data=(x_test, y_test),
        epochs=250,
        batch_size=8)

Train on 120 samples, validate on 30 samples
Epoch 1/250
120/120 [==============================] - 0s 223us/step - loss: 0.8024 - accuracy: 0.7167 - val_loss: 0.7927 - val_accuracy: 0.6667
Epoch 2/250
120/120 [==============================] - 0s 185us/step - loss: 0.7790 - accuracy: 0.6917 - val_loss: 0.7871 - val_accuracy: 0.6333
Epoch 3/250
120/120 [==============================] - 0s 187us/step - loss: 0.7669 - accuracy: 0.6833 - val_loss: 0.7704 - val_accuracy: 0.6333
Epoch 4/250
120/120 [==============================] - 0s 186us/step - loss: 0.7497 - accuracy: 0.6917 - val_loss: 0.7499 - val_accuracy: 0.6667
Epoch 5/250
120/120 [==============================] - 0s 178us/step - loss: 0.7363 - accuracy: 0.6917 - val_loss: 0.7365 - val_accuracy: 0.6333
Epoch 6/250
120/120 [==============================] - 0s 189us/step - loss: 0.7217 - accuracy: 0.7167 - val_loss: 0.7146 - val_accuracy: 0.8333
Epoch 7/250
120/120 [==============================] - 0s 181us/step - loss: 0.7077 -

Epoch 112/250
120/120 [==============================] - 0s 184us/step - loss: 0.1767 - accuracy: 0.9833 - val_loss: 0.1786 - val_accuracy: 0.9000
Epoch 113/250
120/120 [==============================] - 0s 184us/step - loss: 0.1730 - accuracy: 0.9833 - val_loss: 0.1627 - val_accuracy: 0.9667
Epoch 114/250
120/120 [==============================] - 0s 213us/step - loss: 0.1724 - accuracy: 0.9833 - val_loss: 0.1689 - val_accuracy: 0.9667
Epoch 115/250
120/120 [==============================] - 0s 183us/step - loss: 0.1715 - accuracy: 0.9833 - val_loss: 0.1629 - val_accuracy: 0.9667
Epoch 116/250
120/120 [==============================] - 0s 185us/step - loss: 0.1701 - accuracy: 0.9833 - val_loss: 0.1654 - val_accuracy: 0.9667
Epoch 117/250
120/120 [==============================] - 0s 185us/step - loss: 0.1676 - accuracy: 0.9833 - val_loss: 0.1610 - val_accuracy: 0.9667
Epoch 118/250
120/120 [==============================] - 0s 191us/step - loss: 0.1660 - accuracy: 0.9833 - val_loss: 0

Epoch 223/250
120/120 [==============================] - 0s 229us/step - loss: 0.0856 - accuracy: 0.9833 - val_loss: 0.0977 - val_accuracy: 0.9333
Epoch 224/250
120/120 [==============================] - 0s 210us/step - loss: 0.0870 - accuracy: 0.9833 - val_loss: 0.0914 - val_accuracy: 0.9667
Epoch 225/250
120/120 [==============================] - 0s 214us/step - loss: 0.0843 - accuracy: 0.9833 - val_loss: 0.1122 - val_accuracy: 0.9333
Epoch 226/250
120/120 [==============================] - 0s 236us/step - loss: 0.0825 - accuracy: 0.9833 - val_loss: 0.0901 - val_accuracy: 0.9667
Epoch 227/250
120/120 [==============================] - ETA: 0s - loss: 0.0367 - accuracy: 1.00 - 0s 266us/step - loss: 0.0840 - accuracy: 0.9833 - val_loss: 0.0896 - val_accuracy: 0.9667
Epoch 228/250
120/120 [==============================] - 0s 238us/step - loss: 0.0836 - accuracy: 0.9833 - val_loss: 0.1022 - val_accuracy: 0.9333
Epoch 229/250
120/120 [==============================] - 0s 236us/step - los

**Performing Inference**

Let's infer one element of our test dataset and see the probabilities of the network's prediction.

In [16]:
prediction = net.predict(x_test[0].reshape([1,4]))
print("Prediction probabilities:")
for i, category in enumerate(encoder.classes_):
  print(category,": %.2f%%" % (prediction[0][i]*100))

print("Real class: ", encoder.classes_[np.argmax(y_test[0])]) 

Prediction probabilities:
Iris-setosa : 99.87%
Iris-versicolor : 0.13%
Iris-virginica : 0.00%
Real class:  Iris-setosa


**Evaluating the trained neural network**

A better way to evaluate a classification model is by using a confusion matrix. In this matrix we correlate the predicted class with the desired class.

In [17]:
print('Confusion Matrix')
print(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(net.predict(x_test), axis=1)))
print('\nClassification Report')
print(classification_report(np.argmax(y_test, axis=1), np.argmax(net.predict(x_test), axis=1), target_names=encoder.classes_))

Confusion Matrix
[[10  0  0]
 [ 0  9  2]
 [ 0  0  9]]

Classification Report
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       1.00      0.82      0.90        11
 Iris-virginica       0.82      1.00      0.90         9

       accuracy                           0.93        30
      macro avg       0.94      0.94      0.93        30
   weighted avg       0.95      0.93      0.93        30

